In [8]:
import numpy as np
from PIL import Image
import tensorflow as tf

In [25]:
batch_size = 64
img_height = 48
img_width = 48
AUTOTUNE = tf.data.AUTOTUNE

In [4]:
import pathlib
data_dir_train = pathlib.Path('./images/images/train')
data_dir_val = pathlib.Path('./images/images/validation')

In [27]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir_train,
  image_size=(img_height, img_width),
  batch_size=batch_size,
  color_mode='grayscale'
  )

Found 28821 files belonging to 7 classes.


In [28]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir_val,
  image_size=(img_height, img_width),
  batch_size=batch_size,
  color_mode='grayscale'
  )

Found 7066 files belonging to 7 classes.


In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import RandomFlip, RandomRotation, Rescaling, RandomZoom, RandomContrast

rescaling = Sequential([
    Rescaling(1./255),
])

preprocessing_seq = Sequential([
    rescaling,
    RandomFlip(),
    RandomRotation(0.2),
    RandomZoom((-0.3, 0.3)),
    RandomContrast(0.1)
])

In [30]:
new_train_ds = train_ds.map(
    lambda x, y: (preprocessing_seq(x, training=True), y)
)

In [31]:
new_val_ds = val_ds.map(
    lambda x, y: (rescaling(x), y)
)

In [32]:
new_train_ds.prefetch(buffer_size=AUTOTUNE)
new_val_ds.prefetch(buffer_size=AUTOTUNE)

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 48, 48, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [34]:
num_classes = 7

In [46]:
from tensorflow.keras.layers import Conv2D, Input, Dropout, Dense, Lambda, MaxPool2D, Flatten
from tensorflow.nn import local_response_normalization

alexnet = Sequential([
    Input(shape=(48, 48, 1)),
    Conv2D(filters=96, kernel_size=11, strides=4, padding='same', activation='relu'),
    Lambda(local_response_normalization),
    MaxPool2D(pool_size=3, strides=2, padding='same'),

    Conv2D(filters=256, kernel_size=5, strides=4, padding='same', activation='relu'),
    Lambda(local_response_normalization),
    MaxPool2D(pool_size=3, strides=2,  padding='same'),

    Conv2D(filters=384, kernel_size=3, strides=4, padding='same', activation='relu'),
    Dropout(0.2),
    Conv2D(filters=384, kernel_size=3, strides=4, padding='same', activation='relu'),
    Dropout(0.2),
    Conv2D(filters=256, kernel_size=3, strides=4, padding='same', activation='relu'),
    Dropout(0.2),

    Flatten(),
    Dense(units=4096, activation='relu'),
    Dense(units=4096, activation='relu'),
    Dense(units=num_classes, activation='softmax')
])

In [47]:
alexnet.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_50 (Conv2D)          (None, 12, 12, 96)        11712     
                                                                 
 lambda_20 (Lambda)          (None, 12, 12, 96)        0         
                                                                 
 max_pooling2d_20 (MaxPooli  (None, 6, 6, 96)          0         
 ng2D)                                                           
                                                                 
 conv2d_51 (Conv2D)          (None, 2, 2, 256)         614656    
                                                                 
 lambda_21 (Lambda)          (None, 2, 2, 256)         0         
                                                                 
 max_pooling2d_21 (MaxPooli  (None, 1, 1, 256)         0         
 ng2D)                                               

In [48]:
from tensorflow.keras.optimizers import Adam

alexnet.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])